In [244]:
import numpy as np
import math
import pandas as pd 

In [245]:
# Read in input matrix
df = pd.read_csv("matrix3.csv")

### Formatting input

In [246]:
def format_values(df):
    # remove rows with 0 rehearsal duration
    df = df.loc[df.slot_size > 0, :]

    # change cast strings to sets
    df.cast = (df.cast.str.split(", ")).apply(set)

    # round durations to nearest 0.25 of a rehearsal (30 mins)
    df.loc[df.slot_size % 0.25 != 0, "slot_size"] = df.loc[df.slot_size % 0.25 != 0].slot_size.apply(lambda x: math.ceil(x*4)/4)

    # remove trailing whitespaces from rehearsal lead entries
    df.loc[:, "lead"] = df.loc[:, "lead"].str.strip()

    return df

In [247]:
# Rename columns
to_rename = {
    'Song': 'song_name',
    'Duration': 'slot_size',
    'Lead': 'lead',
    'Singers': 'cast',
}
df.rename(columns=to_rename, inplace=True)

# format all col values as required
df = format_values(df)

c:\Users\snuzz\anaconda3\lib\site-packages\pandas\core\generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
c:\Users\snuzz\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


### Separate song slots by rehearsal lead type (e.g. director, choreo, MD)

In [248]:
# Find the types of rehearsals listed (so these can occur simultaneously)
rehearsal_leads = df.lead.unique()
rehearsal_leads

array(['Director', 'Choreo', 'MD'], dtype=object)

In [249]:
def get_full_and_partial_slots(df, rehearsal_lead):
    """Separate out rehearsals by rehearsal lead and full and partial slots (if a slot
    is partial then it needs to be paired with another slot to make a full slot)
    """
    to_rehearse = df[df.lead == rehearsal_lead]
    separated_slots = {
        'full_slots': to_rehearse[to_rehearse.slot_size % 1 == 0],
        'partial_slots': to_rehearse[to_rehearse.slot_size % 1 != 0]
    }
    return separated_slots

In [250]:
def set_overlap(sets):
    for i, set in enumerate(sets):
        if  i == 0:
            set_intersections = set
            set_unions = set
        else:
            set_intersections = set_intersections & set
            set_unions = set_intersections | set
    return len(set_intersections)/len(set_unions)


In [251]:



def calculate_overlap_array(partial_slots, dimensions):
    
    overlap_array = np.zeros([len(partial_slots)]*dimensions)

    for i in range(len(partial_slots)):
        for j in range(len(partial_slots)):
            if i == j:
                overlap_array[i,j] = -1
            else:
                if dimensions == 2:
                    overlap_array[i,j] = set_overlap([partial_slots.loc[i, 'cast'], partial_slots.loc[j, "cast"]])
                else:
                    for k in range(len(partial_slots)):
                        if (i == k) or (j == k):
                            overlap_array[i,j,k] == -1
                        else:
                            if dimensions == 3:
                                overlap_array[i,j,k] = set_overlap([partial_slots.loc[i, 'cast'], partial_slots.loc[j, "cast"], partial_slots.loc[k, "cast"]])
                            else:
                                for l in range(overlap_array.shape[3]):
                                    if (i==l) or (j==l) or (k==l):
                                        overlap_array[i,j,k,l] == -1
                                    else:
                                        overlap_array[i,j,k,l] = set_overlap([partial_slots.loc[i, 'cast'], partial_slots.loc[j, "cast"], partial_slots.loc[k, "cast"], partial_slots.loc[l, "cast"]])
    return(overlap_array)

In [252]:
def find_complementary_durations(song_slot_size, song_name, remaining_slots):
    complementary_durations = [
        x for x in remaining_slots.slot_size.unique() if (x + song_slot_size) % 1 == 0
        ]
    # if can't find pair, sends to bin
    if not complementary_durations:
        print(f"No time match for {song_name}")
    return complementary_durations



In [253]:
def find_best_song_tuple_from_complementary_durations(song_index,
                                                complementary_durations,
                                                partial_slots,
                                                remaining_slots,
                                                overlap_array
                                                ):
    song_sets = []
    overlap_scores = []

    for complementary_duration in complementary_durations:
        possible_matches = remaining_slots[remaining_slots.slot_size == complementary_duration].index
        if list(possible_matches):
            max_overlap = overlap_array.loc[possible_matches, song_index].max()
            best_choice = overlap_array.loc[possible_matches, song_index].idxmax()
            song_sets.append((song_index, best_choice))
            overlap_scores.append(max_overlap)

    if overlap_scores:
        set_with_max_overlap = song_sets[np.argmin(overlap_scores)]
        return set_with_max_overlap
    else:
        return None



In [254]:
def get_set_vals_from_tuple(set_tuple, partial_slots):
    current_set = set()
    duration = 0
    cast = set()
    for element in set_tuple:
        current_set.add(partial_slots.loc[element, "original_index"])
        duration += partial_slots.loc[element, "slot_size"]
        cast = cast | partial_slots.loc[element, "cast"]
    
    out_dict = {
        'pair_indices': current_set,
        'slot_size': duration,
        'cast': cast
    }

    return out_dict


In [257]:
def find_partial_slot_pairings(partial_slots):

    # order songs by cast size (so large numbers are addressed first)
    partial_slots['cast_size'] = partial_slots.cast.apply(len)
    partial_slots = partial_slots.sort_values(by='cast_size', ascending=False).reset_index().rename(columns={'index': 'original_index'})

    # get pair overlap arrays - start with only 2d and make 3 and 4d only if required
    overlap_array = pd.DataFrame(calculate_overlap_array(partial_slots, 2))

    # Create paired and unpaired slot dictionaries

    paired_slots = {
        'pair_indices': [],
        'slot_size': [],
        'cast': []
    }

    unpaired_slots = {
        'song_index': [],
        'slot_size': [],
        'cast': []
    }

    # Create a remain_to_pair list so that only unpaired songs are considered when pairing
    remain_to_pair = list(partial_slots.sort_values(by="slot_size", ascending=False).index)

    while len(remain_to_pair) > 1:
        song_index = remain_to_pair[0]
        remain_to_pair.remove(song_index)
        remaining_slots = partial_slots.loc[remain_to_pair, :]
        original_song_index = partial_slots.loc[song_index, 'original_index']
        song_slot_size = partial_slots.loc[song_index, 'slot_size']
        song_name = partial_slots.loc[song_index, 'song_name']

        complementary_durations = find_complementary_durations(song_slot_size, song_name, remaining_slots)
        # if not complementary durations were found, add this song to the unpaired dict
        if not complementary_durations:
            print(f"{song_name} added to unpaired slots")
            unpaired_slots['song_index'].append(original_song_index)
            unpaired_slots['slot_size'].append(song_slot_size)
            unpaired_slots['cast'].append(partial_slots.loc[song_index, 'cast'])
        else:
            best_set_tuple = find_best_song_tuple_from_complementary_durations(
                song_index, complementary_durations, partial_slots, 
                remaining_slots, overlap_array)
            if best_set_tuple is None:
                print(f"{song_name} added to unpaired slots")
                unpaired_slots['song_index'].append(original_song_index)
                unpaired_slots['slot_size'].append(song_slot_size)
                unpaired_slots['cast'].append(partial_slots.loc[song_index, 'cast'])
            else:
                set_tuple_dict = get_set_vals_from_tuple(best_set_tuple, partial_slots)
                if set_tuple_dict['pair_indices'] in paired_slots['pair_indices']:
                    pass
                else:
                    for key in paired_slots:
                        paired_slots[key].append(set_tuple_dict[key])
                    for elem in best_set_tuple:
                        if elem in remain_to_pair:
                            remain_to_pair.remove(elem)

    if len(remain_to_pair) == 1:
        song_index = remain_to_pair[0]
        original_song_index = partial_slots.loc[song_index, 'original_index']
        song_slot_size = partial_slots.loc[song_index, 'slot_size']
        song_name = partial_slots.loc[song_index, 'song_name']
        print(f"{song_name} added to unpaired slots")
        unpaired_slots['song_index'].append(original_song_index)
        unpaired_slots['slot_size'].append(song_slot_size)
        unpaired_slots['cast'].append(partial_slots.loc[song_index, 'cast'])


    out_dict = {
        'paired': pd.DataFrame.from_dict(paired_slots),
        'unpaired': pd.DataFrame.from_dict(unpaired_slots)
    }

    return out_dict

            
                







In [258]:

rehearsals = {}
for rehearsal_lead in rehearsal_leads:
    separated_slots = get_full_and_partial_slots(df, rehearsal_lead)

    # full slots are fine, need to find pairings for partial slots
    rehearsals[rehearsal_lead] = find_partial_slot_pairings(separated_slots['partial_slots'])
    rehearsals[rehearsal_lead]['full_slots'] = separated_slots['full_slots']




No time match for Opening Night Blocking
Opening Night Blocking added to unpaired slots
No time match for Goodbye Blocking
Goodbye Blocking added to unpaired slots
No time match for Hard to be the Bard Blocking
Hard to be the Bard Blocking added to unpaired slots
No time match for Why Cause I'm A Guy Blocking
Why Cause I'm A Guy Blocking added to unpaired slots
No time match for Big Spender All Blocking
Big Spender All Blocking added to unpaired slots
When He Sees Me Blocking added to unpaired slots
No time match for Coffee Break Choreo
Coffee Break Choreo added to unpaired slots
No time match for Meet the Plastics All Choreo
Meet the Plastics All Choreo added to unpaired slots
Meet the Plastics Trio Only Choreo added to unpaired slots


In [259]:
for lead in rehearsals:
    print(lead)
    for key in rehearsals[lead]:
        print(key)
        display(rehearsals[lead][key])

Director
paired


,pair_indices,slot_size,cast
0,"{1, 2}",2.0,"{Ellen McDermott, Lexi Dowle, Lucy Ella-Foster..."
1,"{4, 31}",2.0,"{Sonia Huntley-Robertson, Lindsey McBratney, E..."


unpaired


,song_index,slot_size,cast
0,0,0.75,"{Sonia Huntley-Robertson, Lindsey McBratney, E..."
1,6,0.75,"{Sonia Huntley-Robertson, Lindsey McBratney, E..."
2,14,0.75,"{Sophie Coad, Katharine Mann, Ellie Morrow, Ha..."
3,21,0.75,"{Bertie Chatfield, Joey Chan, Graham Broad, He..."
4,26,0.75,"{Sonia Huntley-Robertson, Katharine Mann, Holl..."
5,12,0.75,"{Sonia Huntley-Robertson, Katharine Mann, Soph..."


full_slots


...

,pair_indices,slot_size,cast
0,"{30, 23}",1.0,"{Ellen McDermott, Kelly Brewer, Lexi Dowle, He..."
1,"{25, 36}",1.0,"{Sonia Huntley-Robertson, Holly Thompson, Lind..."
2,"{17, 37}",1.0,"{Hannah Dare, Holly Thompson, Lindsey McBratne..."
3,"{48, 53}",1.0,"{Lucy Ella-Foster, Evan Huntley-Robertson, Eve..."
4,"{16, 51}",1.0,"{Bertie Chatfield, Joey Chan, Emma Morris, Ell..."
5,"{13, 7}",1.0,"{Sonia Huntley-Robertson, Lindsey McBratney, E..."
6,"{19, 52}",1.0,"{Bertie Chatfield, Joey Chan, Graham Broad, He..."
7,"{50, 11}",1.0,"{Sonia Huntley-Robertson, Katharine Mann, Soph..."
8,"{43, 20}",1.0,"{Ellie Morrow, Kelly Brewer, Lexi Dowle, Phili..."
9,"{24, 54}",1.0,"{Sonia Huntley-Robertson, Katharine Mann, Holl..."


unpaired


,song_index,slot_size,cast


full_slots


,song_name,slot_size,lead,cast
9,Hello Singing,1.0,MD,"{Sophie Coad, Katharine Mann, Ellie Morrow, Ha..."
29,KONY Solo Dancer Singing,1.0,MD,"{Ellie Morrow, Hannah Dare, Holly Thompson, Ke..."
38,When I Grow Up inc Soloists Singing,1.0,MD,"{Lindsey McBratney, Ellen McDermott, Lucy Ella..."
39,Welcome to Wonderland All Singers Singing,1.0,MD,"{Sonia Huntley-Robertson, Katharine Mann, Bert..."
40,Welcome to Wonderland Soloists Central Solos O...,1.0,MD,"{Sonia Huntley-Robertson, Bertie Chatfield, Jo..."
44,On the Highway of Love Soloists Singing,1.0,MD,"{Evan Huntley-Robertson, Emma Culley, Tim Beas..."


In [227]:
rehearsals['Director']['unpaired']

,song_index,slot_size,cast


In [430]:
song_pairings['Director']['paired'].to_csv('ex.csv')

In [426]:
song_pairings['Choreo']['full_slots']

,set,duration,cast
3,TWBDT Choreo,3.0,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
8,Freak Flag Choreo,3.0,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
10,Hello Choreo,1.0,"{Holly Thompson, Kiah Whitehead, Laura Boswell..."
15,School Song Dancing,4.0,"{Holly Thompson, Evan Huntley-Robertson, Sharo..."
18,Nowadays Hot Honey Rag Dancing,4.0,"{Sharon Forsyth, Aimee Burton, Hannah Dare, Ph..."
22,Candy Store Choreo,2.0,"{Lexi Dowle, Heather Bokota, Philippa Wilding,..."
28,KONY Dancing,4.0,"{Holly Thompson, Evan Huntley-Robertson, Steph..."
35,Welcome to Wonderland Choreo,3.0,"{Eveey Ruth, Katharine Mann, Graham Broad, Son..."
41,Footloose Dancing and Backing Shouts,4.0,"{Holly Thompson, Evan Huntley-Robertson, Hanna..."
42,Good Morning Dancing,4.0,"{Evan Huntley-Robertson, Ellie Morrow, Hannah ..."


In [374]:
df1

,Song,Duration,Director,Singers
0,Opening Night Blocking,0.75,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
1,Drink With Me Blocking,0.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
2,Hard Knock Life Blocking,1.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
3,TWBDT Choreo,3.00,Choreo,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
4,Once We Were Kings Blocking,1.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
5,Finale B Blocking,1.00,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
6,Goodbye Blocking,0.75,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
7,Freak Flag Singing,0.50,MD,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
8,Freak Flag Choreo,3.00,Choreo,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
9,Hello Singing,1.00,MD,"{Holly Thompson, Kiah Whitehead, Laura Boswell..."


In [348]:
df1

,Song,Duration,Director,Singers
0,Opening Night Blocking,0.75,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
1,Drink With Me Blocking,0.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
2,Hard Knock Life Blocking,1.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
3,TWBDT Choreo,3.00,Choreo,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
4,Once We Were Kings Blocking,1.50,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
5,Finale B Blocking,1.00,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
6,Goodbye Blocking,0.75,Director,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
7,Freak Flag Singing,0.50,MD,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
8,Freak Flag Choreo,3.00,Choreo,"{Colin Bousfield, Holly Thompson, Dave Pound, ..."
9,Hello Singing,1.00,MD,"{Holly Thompson, Kiah Whitehead, Laura Boswell..."


In [245]:
songs_left_to_group = list(partial_slots.sort_values(by="Duration", ascending=False).index)

In [249]:
songs_left = partial_slots.loc[partial_slots.index.isin(songs_left_to_group)]
songs_left[songs_left.Duration == 0.25]

,Song,Duration,Director,Singers,cast_size
2,KONY Chorus Singing,0.25,MD,"{Patrick Coad, Eveey Ruth, Tim Beasley, Ellen ...",13
8,On the Highway of Love Backing Singing,0.25,MD,"{Patrick Coad, Eveey Ruth, Holly Thompson, Gra...",10
11,School Song Dancer Singing,0.25,MD,"{Holly Thompson, Evan Huntley-Robertson, Sharo...",8
15,Hard Knock Life Selected Soloists,0.25,MD,"{Eveey Ruth, Laura Boswell, Lucy Ella-Foster}",3
17,Freak Flag Minor Solos Singing,0.25,MD,"{Emma Morris, Ellen McDermott, Lynda Lawrence}",3


In [243]:
partial_slots.loc[song_index, "Duration"]

0.75

In [241]:
[(i, j) for (i in partial_slots.Duration.unique()) and (j in partial_slots.Duration.unique()) if (i + j + partial_slots.loc[song_index, "Duration"]) % 1 == 0]

SyntaxError: expected 'else' after 'if' expression (2682638195.py, line 1)

In [236]:
song_index = 1
partial_slots.loc[song_index, "Duration"]

0.75

In [237]:
partial_slots.Duration.unique()

array([0.5 , 0.75, 0.25])

[0.25]

In [193]:
pd.DataFrame(overlap_array)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,-1.000000,0.425532,0.276596,0.255319,0.234043,0.234043,0.234043,0.212766,0.212766,0.191489,0.170213,0.170213,0.148936,0.085106,0.063830,0.063830,0.063830,0.063830,0.042553,0.042553
1,0.425532,-1.000000,0.222222,0.185185,0.192308,0.148148,0.550000,0.071429,0.071429,0.450000,0.120000,0.166667,0.173913,0.142857,0.095238,0.045455,0.045455,0.000000,0.100000,0.100000
2,0.276596,0.222222,-1.000000,0.190476,0.846154,0.142857,0.200000,0.150000,0.150000,0.100000,0.235294,0.000000,0.176471,0.133333,0.066667,0.142857,0.066667,0.142857,0.071429,0.071429
3,0.255319,0.185185,0.190476,-1.000000,0.150000,0.210526,0.000000,0.100000,0.100000,0.312500,0.428571,0.333333,0.000000,0.000000,0.250000,0.071429,0.071429,0.000000,0.000000,0.000000
4,0.234043,0.192308,0.846154,0.150000,-1.000000,0.100000,0.157895,0.105263,0.105263,0.111111,0.187500,0.000000,0.200000,0.153846,0.076923,0.166667,0.076923,0.166667,0.083333,0.083333
5,0.234043,0.148148,0.142857,0.210526,0.100000,-1.000000,0.157895,0.235294,0.235294,0.052632,0.117647,0.117647,0.058824,0.071429,0.076923,0.076923,0.076923,0.076923,0.181818,0.181818
6,0.234043,0.550000,0.200000,0.000000,0.157895,0.157895,-1.000000,0.105263,0.105263,0.000000,0.000000,0.000000,0.285714,0.153846,0.000000,0.000000,0.076923,0.000000,0.181818,0.181818
7,0.212766,0.071429,0.150000,0.100000,0.105263,0.235294,0.105263,-1.000000,1.000000,0.000000,0.125000,0.058824,0.133333,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.090909
8,0.212766,0.071429,0.150000,0.100000,0.105263,0.235294,0.105263,1.000000,-1.000000,0.000000,0.125000,0.058824,0.133333,0.000000,0.000000,0.083333,0.083333,0.083333,0.000000,0.090909
9,0.191489,0.450000,0.100000,0.312500,0.111111,0.052632,0.000000,0.000000,0.000000,-1.000000,0.214286,0.307692,0.000000,0.083333,0.200000,0.090909,0.000000,0.000000,0.000000,0.000000


In [192]:
0.75 % 0.5

0.25

In [156]:
len(set_i & set_j)

2

In [157]:
len(set_i | set_j)

2

In [149]:
partial_slots

,Song,Duration,Director,Singers,cast_size
0,Freak Flag Singing,0.50,MD,"{Colin Bousfield, Holly Thompson, Dave Pound, ...",47
1,Michael in the Bathroom Chorus Singing and Blo...,0.75,MD,"{Colin Bousfield, Dave Pound, Graham Broad, Ph...",20
2,KONY Chorus Singing,0.25,MD,"{Patrick Coad, Eveey Ruth, Tim Beasley, Ellen ...",13
3,Hard to be the Bard Backing Singing,0.40,MD,"{Holly Thompson, Kiah Whitehead, Laura Boswell...",12
4,Coffee Break no Solos Singing,0.75,MD,"{Eveey Ruth, Tim Beasley, Ellen McDermott, Ber...",11
5,Something's Coming Unison All Singing,0.75,MD,"{Holly Thompson, Graham Broad, Kiah Whitehead,...",11
6,Why Cause I'm A Guy Singing,0.50,MD,"{Colin Bousfield, Patrick Coad, Dave Pound, Gr...",11
7,Big Spender All Singing,0.50,MD,"{Patrick Coad, Holly Thompson, Eveey Ruth, Gra...",10
8,On the Highway of Love Backing Singing,0.25,MD,"{Patrick Coad, Eveey Ruth, Holly Thompson, Gra...",10
9,Candy Store All Singing,0.50,MD,"{Lexi Dowle, Heather Bokota, Philippa Wilding,...",9


In [151]:
print(np.ones((len(partial_slots), len(partial_slots))))

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.